# Imports

In [1]:
from qiskit.circuit.library.standard_gates import RXGate, RZGate, RYGate, CXGate, CZGate, SGate, HGate
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from typing import Dict, Tuple, List
import numpy as np
from numpy import linalg as LA
from qiskit.opflow import X, Z, I, H, Y
from qiskit.quantum_info import Pauli
import matplotlib
from qiskit import assemble, Aer
from qiskit import IBMQ
from qiskit import *
from qiskit.visualization import plot_histogram
import math
from qiskit.algorithms.optimizers import L_BFGS_B, COBYLA

# Implementing the ansatz from qiskit tutorial

In [2]:
def calc_linear_entangelment_ansatz(num_of_qubits, thetas, depth):
    if thetas.size != 2*num_of_qubits*(depth + 1):
        print("ERROR with thetas size")
    else:
        qr = QuantumRegister(num_of_qubits, name="qubit")
        qc = QuantumCircuit(qr)
        
        for iteration in range(depth):
            for qubit_index in range(num_of_qubits):
                RY_theta_index = iteration*2*num_of_qubits + qubit_index
                RZ_theta_index = RY_theta_index + num_of_qubits
                qc.append(RYGate(thetas[RY_theta_index]), [qr[qubit_index]])
                qc.append(RZGate(thetas[RZ_theta_index]), [qr[qubit_index]])
            for qubit_index in range(num_of_qubits - 1):
                qc.append(CXGate(), [qr[qubit_index], qr[qubit_index + 1]])
        
        for qubit_index in range(num_of_qubits):
                RY_theta_index = 2*num_of_qubits*depth + qubit_index
                RZ_theta_index = RY_theta_index + num_of_qubits
                qc.append(RYGate(thetas[RY_theta_index]), [qr[qubit_index]])
                qc.append(RZGate(thetas[RZ_theta_index]), [qr[qubit_index]])
        
        return qc

def calc_full_entangelment_ansatz(num_of_qubits, thetas, depth):
    if thetas.size != 2*num_of_qubits*(depth + 1):
        print("ERROR with thetas size")
    else:
        qr = QuantumRegister(num_of_qubits, name="qubit")
        qc = QuantumCircuit(qr)
        
        for iteration in range(depth):
            for qubit_index in range(num_of_qubits):
                RY_theta_index = iteration*2*num_of_qubits + qubit_index
                RZ_theta_index = RY_theta_index + num_of_qubits
                qc.append(RYGate(thetas[RY_theta_index]), [qr[qubit_index]])
                qc.append(RZGate(thetas[RZ_theta_index]), [qr[qubit_index]])
            for qubit_index in range(num_of_qubits - 1):
                for target_qubit_index in range(qubit_index + 1, num_of_qubits):
                    qc.append(CXGate(), [qr[qubit_index], qr[target_qubit_index]])
        
        for qubit_index in range(num_of_qubits):
                RY_theta_index = 2*num_of_qubits*depth + qubit_index
                RZ_theta_index = RY_theta_index + num_of_qubits
                qc.append(RYGate(thetas[RY_theta_index]), [qr[qubit_index]])
                qc.append(RZGate(thetas[RZ_theta_index]), [qr[qubit_index]])
        
        return qc

def calc_ansatz_circ(num_of_qubits, entangelment, thetas, depth):
    if entangelment == "linear":
        return calc_linear_entangelment_ansatz(num_of_qubits, thetas, depth)
    assert entangelment == "full", "entangelment has to be linear or full"
    return calc_full_entangelment_ansatz(num_of_qubits, thetas, depth)

# Generating the hamiltonians

In [3]:
H2_molecule_Hamiltonian_2_qubits =  -0.5053051899926562*(I^I) + \
                                    -0.3277380754984016*(Z^I) + \
                                    0.15567463610622564*(Z^Z) + \
                                    -0.3277380754984016*(I^Z)

H2_molecule_Hamiltonian_4_qubits =  -0.8105479805373279 * (I^I^I^I) \
                                    + 0.1721839326191554 * (I^I^I^Z) \
                                    - 0.22575349222402372 * (I^I^Z^I) \
                                    + 0.17218393261915543 * (I^Z^I^I) \
                                    - 0.2257534922240237 * (Z^I^I^I) \
                                    + 0.12091263261776627 * (I^I^Z^Z) \
                                    + 0.16892753870087907 * (I^Z^I^Z) \
                                    + 0.045232799946057826 * (Y^Y^Y^Y) \
                                    + 0.045232799946057826 * (X^X^Y^Y) \
                                    + 0.045232799946057826 * (Y^Y^X^X) \
                                    + 0.045232799946057826 * (X^X^X^X) \
                                    + 0.1661454325638241 * (Z^I^I^Z) \
                                    + 0.1661454325638241 * (I^Z^Z^I) \
                                    + 0.17464343068300453 * (Z^I^Z^I) \
                                    + 0.12091263261776627 * (Z^Z^I^I)

LiH_molecule_hamiltonian =  -7.49894690201071*(I^I^I^I) + \
                            -0.0029329964409502266*(X^X^Y^Y) + \
                            0.0029329964409502266*(X^Y^Y^X) + \
                            0.01291078027311749*(X^Z^X^I) + \
                            -0.0013743761078958677*(X^Z^X^Z) + \
                            0.011536413200774975*(X^I^X^I) + \
                            0.0029329964409502266*(Y^X^X^Y) + \
                            -0.0029329964409502266*(Y^Y^X^X) + \
                            0.01291078027311749*(Y^Z^Y^I) + \
                            -0.0013743761078958677*(Y^Z^Y^Z) + \
                            0.011536413200774975*(Y^I^Y^I) + \
                            0.16199475388004184*(Z^I^I^I) + \
                            0.011536413200774975*(Z^X^Z^X) + \
                            0.011536413200774975*(Z^Y^Z^Y) + \
                            0.12444770133137588*(Z^Z^I^I) + \
                            0.054130445793298836*(Z^I^Z^I) + \
                            0.05706344223424907*(Z^I^I^Z) + \
                            0.012910780273117487*(I^X^Z^X) + \
                            -0.0013743761078958677*(I^X^I^X) + \
                            0.012910780273117487*(I^Y^Z^Y) + \
                            -0.0013743761078958677*(I^Y^I^Y) + \
                            0.16199475388004186*(I^Z^I^I) + \
                            0.05706344223424907*(I^Z^Z^I) + \
                            0.054130445793298836*(I^Z^I^Z) + \
                            -0.013243698330265966*(I^I^Z^I) + \
                            0.08479609543670981*(I^I^Z^Z) + \
                            -0.013243698330265952*(I^I^I^Z) 

def create_pauli_string_with_pauli_op_on_index_i(pauli_op, i, qubits_num):
    if i == 1:
        pauli_string = pauli_op
        for qubit in range(qubits_num - 1):
            pauli_string = pauli_string ^ I
        return pauli_string
    
    pauli_string = I
    for qubit in range(2, qubits_num + 1):
        if qubit == i:
            pauli_string = pauli_string ^ pauli_op
        else:
            pauli_string = pauli_string ^ I
            
    return pauli_string

def create_pauli_string_with_pauli_ops_on_index_i_and_j(pauli_op_second, i, pauli_op_first, j, qubits_num):
    if j == 1:
        pauli_string = pauli_op_first
        for qubit in range(2, qubits_num + 1):
            if qubit == i:
                pauli_string = pauli_string ^ pauli_op_second
            else:
                pauli_string = pauli_string ^ I
        return pauli_string
    
    pauli_string = I
    for qubit in range(2, qubits_num + 1):
        if qubit == j:
            pauli_string = pauli_string ^ pauli_op_first
        elif qubit == i:
            pauli_string = pauli_string ^ pauli_op_second
        else:
            pauli_string = pauli_string ^ I
            
    return pauli_string
    
def get_Ising_model_hamiltonian():
    hamiltonian = I
    for qubit in range(QUBITS_NUM - 1):
        hamiltonian = hamiltonian^I
    hamiltonian = 0 * hamiltonian
    
    for i in range(1, QUBITS_NUM + 1):
        a_i = np.random.random_sample()
        x_i = create_pauli_string_with_pauli_op_on_index_i(X, i, QUBITS_NUM)
        hamiltonian = hamiltonian + a_i * x_i
        for j in range(1, i):
            J_ij = np.random.random_sample()
            z_ij = create_pauli_string_with_pauli_ops_on_index_i_and_j(Z, i, Z, j, QUBITS_NUM)
            hamiltonian = hamiltonian + J_ij * z_ij
    
    return hamiltonian
    
def get_hamiltonian(hamiltonian_type):
    if hamiltonian_type == "Ising Model":
        return get_Ising_model_hamiltonian()
    elif hamiltonian_type == "H2":
        if QUBITS_NUM == 2:
            return H2_molecule_Hamiltonian_2_qubits
        elif QUBITS_NUM == 4:
            return H2_molecule_Hamiltonian_4_qubits
        else:
            print("Error: H2 dosen't have the hamiltonian for this amount of qubits:")
            print(QUBITS_NUM)
    elif hamiltonian_type == "LiH":
        return LiH_molecule_hamiltonian
    else:
        print("Error: no support for this type of hamiltonian:")
        print(hamiltonian_type)

# Expectation Value

In [4]:
reducing_to_pauli_z_dict = {
    Pauli('I'): Pauli('I'),
    Pauli('Z'): Pauli('Z'),
    Pauli('X'): Pauli('Z'),
    Pauli('Y'): Pauli('Z')
} 

def transfrom_hamiltonian_into_pauli_string(hamiltonian):
    pauli_operators = hamiltonian.to_pauli_op().settings['oplist']
    pauli_strings = list(map(lambda pauli_operator: pauli_operator.primitive, pauli_operators))
    pauli_coeff = list(map(lambda pauli_operator: pauli_operator.coeff, pauli_operators))
    return (pauli_strings, pauli_coeff)

def reduce_pauli_matrixes_into_sigma_z(pauli_string):
    for matrix_index in range(QUBITS_NUM):
        pauli_matrix = pauli_string[matrix_index]
        pauli_string[matrix_index].insert(reducing_to_pauli_z_dict[pauli_matrix])
    
    print(pauli_string)
    return pauli_string

def get_z_reduction_for_pauli_string(qc, pauli_string):
    qr = QuantumRegister(QUBITS_NUM, name="q")
    exdend_qc = QuantumCircuit(qr)
    pauli_string = str(pauli_string)
    for qubit_index, pauli_matrix in enumerate(pauli_string):
        if pauli_matrix == "X":
            exdend_qc.append(HGate(), [qr[qubit_index]])
        elif pauli_matrix == "Y":
            exdend_qc.append(HGate(), [qr[qubit_index]])
            exdend_qc.append(SGate(), [qr[qubit_index]])
    qc = qc.compose(exdend_qc)
    return qc

def get_probability_distribution(counts: Dict) -> Dict:
    proba_distribution = {state: (count / NUM_SHOTS) for state, count in counts.items()}
    return proba_distribution

def calculate_probabilities_of_measurments_in_computational_basis(quantum_state_circuit) -> Dict:
    quantum_state_circuit.measure_all()

    transpiled_quantum_state_circuit = transpile(quantum_state_circuit, BACKEND) 
    
    result = BACKEND.run(transpiled_quantum_state_circuit).result() 
    counts = result.get_counts(transpiled_quantum_state_circuit)
    
    return get_probability_distribution(counts)

def sort_probas_dict_by_qubits_string_keys(proba_distribution: Dict) -> Dict:
    return dict(sorted(proba_distribution.items()))

def reset_power_of_minus_1(power_of_minus_1):
    power_of_minus_1 = 0
    return power_of_minus_1

def calculate_expectation_value_of_pauli_string_by_measurments_probas(pauli_string, probas_distribution):
    pauli_string_expectation_value = 0
    power_of_minus_1 = 0
    
    sorted_probas_distribuition = sort_probas_dict_by_qubits_string_keys(probas_distribution)
    for qubits_string, proba in sorted_probas_distribuition.items():
        for string_index in range(QUBITS_NUM):
            if(str(qubits_string[string_index])=="1" and str(pauli_string[string_index])=="Z"):
                power_of_minus_1 += 1
            
        pauli_string_expectation_value += pow(-1, power_of_minus_1)*proba
        power_of_minus_1 = reset_power_of_minus_1(power_of_minus_1)
        
    return pauli_string_expectation_value

def get_expectation_value(pauli_string, probas_distribution):
        return calculate_expectation_value_of_pauli_string_by_measurments_probas(
                                                                                pauli_string, probas_distribution)

# Calculating the target function

In [5]:
def target_function(thetas):
    ansatz = calc_ansatz_circ(QUBITS_NUM, Entangelment, thetas, Depth)
    
    qr = QuantumRegister(QUBITS_NUM, name="qubit")
    qc = QuantumCircuit(qr)
    qc.initialize(Init_state, qc.qubits)
    
    total_expectation_value = 0
    pauli_strings, pauli_coeffs = transfrom_hamiltonian_into_pauli_string(Ham)
    for pauli_index, pauli_string in enumerate(pauli_strings):
        qc_per_pauli_string = qc.compose(ansatz)
        qc_per_pauli_string = get_z_reduction_for_pauli_string(qc_per_pauli_string, pauli_string)
        probas_distribution = calculate_probabilities_of_measurments_in_computational_basis(qc_per_pauli_string)
        total_expectation_value += pauli_coeffs[pauli_index] * get_expectation_value(pauli_string, probas_distribution)
    print("total expectation value:")
    print(total_expectation_value)
    return total_expectation_value
        

# Calculate the difference between the algorithm result and actual result

In [6]:
def calc_diff_from_actual_result(hamiltonian, algorithm_result):
    matrix = hamiltonian.to_matrix()
    eigen_values, eigen_vectors = LA.eig(matrix)
    actual_result = min(eigen_values)
    print("The actual minimal energy is:")
    print(actual_result)
    diff = algorithm_result - actual_result
    print("The differnce between the actual value and the algorithm result is:")
    print(diff)

# Implementing the VQE algorithem

In [7]:
def VQE(optimazer, initial_thetas):
    print("The Hamiltonian running VQE on is:")
    print(Ham)
    point, value, nfev = optimazer.optimize(QUBITS_NUM*2*(Depth + 1), target_function, initial_point=initial_thetas)
    base_energy = value
    print("The VQE result for the minimal energy of the Hamiltonian given is:")
    print(base_energy)
    calc_diff_from_actual_result(Ham, base_energy)

# Setting the simulator and optimazer

In [8]:
bfgs_optimizer = L_BFGS_B(maxiter=2)
cobyla_optimazer = COBYLA(maxiter=50)

provider = IBMQ.enable_account('4cd532424f249f20233857b3b211eb28dfc0e790386bd2ea14d3e0d03c867dfcfec4c2c968e4693f1c9caf7b3f6fad3f6a6393065fb45719692fd1a5177536cb')
real_backend = provider.get_backend('ibmq_belem')
simulator_backend = Aer.get_backend('aer_simulator') 

# VQE with Transverse Ising Model Hamiltonian(4 Qubits) + LINEAR Entangelment Ansatz

In [9]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 4
Ham = get_hamiltonian("Ising Model")
Entangelment = "linear"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)
    

The Hamiltonian running VQE on is:
0.0 * IIII
+ 0.08936700311619616 * XIII
+ 0.05821528908798457 * IXII
+ 0.8222947954335467 * ZZII
+ 0.1523566328069712 * IIXI
+ 0.41740717004270533 * ZIZI
+ 0.3731456072626391 * IZZI
+ 0.3788318574112455 * IIIX
+ 0.8622534518076894 * ZIIZ
+ 0.29594019557918694 * IZIZ
+ 0.7141512435242652 * IIZZ
total expectation value:
0.4335197647126013
total expectation value:
0.424073578650472
total expectation value:
0.5182050830159999
total expectation value:
0.36208273393901086
total expectation value:
0.4584117630197294
total expectation value:
0.4986201958654229
total expectation value:
0.42384959388613946
total expectation value:
0.5099040497124373
total expectation value:
0.5432803330471934
total expectation value:
0.5103361599776894
total expectation value:
0.5081050110468066
total expectation value:
0.37484414442695774
total expectation value:
0.41782245023674597
total expectation value:
0.5086080425496848
total expectation value:
0.5043104165823478
total e

total expectation value:
0.5312435631427306
total expectation value:
0.3639758030338381
total expectation value:
0.5108662620169382
total expectation value:
0.4533345874381753
total expectation value:
0.4882795139420699
total expectation value:
0.463211630105095
total expectation value:
0.4664926634515642
total expectation value:
0.53371343187997
total expectation value:
0.49057284489929787
total expectation value:
0.4860392176746482
total expectation value:
0.46876253944054636
total expectation value:
0.40291890410004116
total expectation value:
0.5055374453455366
total expectation value:
0.4966994675873235
total expectation value:
0.4884231542911225
total expectation value:
0.4285378931667028
total expectation value:
0.44580904621389966
total expectation value:
0.5131703122602973
total expectation value:
0.41333951400038565
total expectation value:
0.510972583331241
total expectation value:
0.4857424678160403
total expectation value:
0.514561608225942
total expectation value:
0.53525

total expectation value:
0.4791864583416771
total expectation value:
0.37473970121230576
total expectation value:
0.42937033282019627
total expectation value:
0.5230894322420758
total expectation value:
0.516959683933155
total expectation value:
0.45120783169476053
total expectation value:
0.4590668437368738
total expectation value:
0.5120374043714172
total expectation value:
0.39872311402815436
total expectation value:
0.4091760488949242
total expectation value:
0.4974873442676604
total expectation value:
0.44974562375356353
total expectation value:
0.4351179179396911
total expectation value:
0.5451920137761721
total expectation value:
0.4302280587234876
total expectation value:
0.4956873461191943
total expectation value:
0.46116056038086395
total expectation value:
0.4045413324210186
total expectation value:
0.4617207068754237
total expectation value:
0.5405244989631223
total expectation value:
0.4767333965458426
total expectation value:
0.47975534087569205
total expectation value:
0

total expectation value:
0.46881649515769946
total expectation value:
0.5098826476355389
total expectation value:
0.4913597074211636
total expectation value:
0.49456451008127394
total expectation value:
0.4464479799055124
total expectation value:
0.47939334529581246
total expectation value:
0.47625863413630437
total expectation value:
0.5035592763892169
total expectation value:
0.39728341454077687
total expectation value:
0.5779309590946817
total expectation value:
0.4009129054754638
total expectation value:
0.407559449321199
total expectation value:
0.5332405907248101
total expectation value:
0.526405619602528
total expectation value:
0.4766293583254798
total expectation value:
0.44303188582522074
total expectation value:
0.41446600346456075
total expectation value:
0.5644509608579424
total expectation value:
0.5131966158499404
total expectation value:
0.5325610202339965
total expectation value:
0.4944432698738984
total expectation value:
0.5455840249574933
total expectation value:
0.

# VQE with Transverse Ising Model Hamiltonian(4 Qubits) + FULL Entangelment Ansatz

In [10]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 4
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)

The Hamiltonian running VQE on is:
0.0 * IIII
+ 0.08936700311619616 * XIII
+ 0.05821528908798457 * IXII
+ 0.8222947954335467 * ZZII
+ 0.1523566328069712 * IIXI
+ 0.41740717004270533 * ZIZI
+ 0.3731456072626391 * IZZI
+ 0.3788318574112455 * IIIX
+ 0.8622534518076894 * ZIIZ
+ 0.29594019557918694 * IZIZ
+ 0.7141512435242652 * IIZZ
total expectation value:
0.7064426361752552
total expectation value:
0.7759606801003268
total expectation value:
0.7183439595989942
total expectation value:
0.6511545456427604
total expectation value:
0.7499394589542984
total expectation value:
0.6992803362273149
total expectation value:
0.7195732032356841
total expectation value:
0.7114403149712291
total expectation value:
0.7040517467659222
total expectation value:
0.7306623943847111
total expectation value:
0.7223755619967517
total expectation value:
0.7479820294034795
total expectation value:
0.7656635703146484
total expectation value:
0.7112080957285121
total expectation value:
0.8064422961544786
total expe

total expectation value:
0.7285575247657187
total expectation value:
0.7575120486058272
total expectation value:
0.6967845499115507
total expectation value:
0.7244730255999281
total expectation value:
0.7394117002653977
total expectation value:
0.6691727772601231
total expectation value:
0.7968696500720952
total expectation value:
0.7434498497755087
total expectation value:
0.7803516191771612
total expectation value:
0.7816539884463785
total expectation value:
0.6673931113812012
total expectation value:
0.7437651339492426
total expectation value:
0.7061397611296107
total expectation value:
0.7074487892616276
total expectation value:
0.690060382176567
total expectation value:
0.6781279392977091
total expectation value:
0.7512431298221406
total expectation value:
0.7735509107299164
total expectation value:
0.7374986866394914
total expectation value:
0.667616198056429
total expectation value:
0.7137929721447288
total expectation value:
0.6741378834708049
total expectation value:
0.7199645

total expectation value:
0.6952617308168187
total expectation value:
0.7507650551771187
total expectation value:
0.7396706856039146
total expectation value:
0.6958034125578588
total expectation value:
0.7990837325397324
total expectation value:
0.6726879180842975
total expectation value:
0.6999676008301203
total expectation value:
0.7375701145597138
total expectation value:
0.7456484113415994
total expectation value:
0.7192573792914203
total expectation value:
0.7983836694731494
total expectation value:
0.8013328459091152
total expectation value:
0.7671681467904875
total expectation value:
0.6481278767715118
total expectation value:
0.7165064574079796
total expectation value:
0.6786544534585182
total expectation value:
0.7674994014226574
total expectation value:
0.6541456300059997
total expectation value:
0.6906701246900009
total expectation value:
0.683011134905202
total expectation value:
0.7289784005766735
total expectation value:
0.7430014037842818
total expectation value:
0.694617

total expectation value:
0.7128363535591542
total expectation value:
0.7963774800822772
total expectation value:
0.7183536400015939
total expectation value:
0.712046334662771
total expectation value:
0.7412705649112503
total expectation value:
0.6483047199091698
total expectation value:
0.6630250527800022
total expectation value:
0.7385140790330761
total expectation value:
0.8130684894903358
total expectation value:
0.7764441367249433
total expectation value:
0.7771750878161788
total expectation value:
0.7149588030503106
total expectation value:
0.6310624816457862
total expectation value:
0.7046787452493661
total expectation value:
0.8051759496338152
total expectation value:
0.7453384433931547
total expectation value:
0.7271771288808225
total expectation value:
0.815764014091596
total expectation value:
0.7385918344583937
total expectation value:
0.7843353249723621
total expectation value:
0.7103192496743038
total expectation value:
0.8115453508956909
total expectation value:
0.7244672

total expectation value:
0.7436798086429238
total expectation value:
0.6670843080293127
total expectation value:
0.782655000611375
total expectation value:
0.6918864422786952
total expectation value:
0.7948088613236337
total expectation value:
0.6818246812046715
total expectation value:
0.733740781428118
total expectation value:
0.7309689280293923
total expectation value:
0.6878990151621489
total expectation value:
0.6633099640565007
total expectation value:
0.742565560923102
total expectation value:
0.740625105241691
total expectation value:
0.7278202904478966
total expectation value:
0.6812656575206384
total expectation value:
0.722093613473445
total expectation value:
0.6894177872991485
total expectation value:
0.7790830614754097
total expectation value:
0.7404584742859992
total expectation value:
0.6718390973686243
total expectation value:
0.6816490650257135
total expectation value:
0.6993096517595327
total expectation value:
0.750662421375978
total expectation value:
0.66459891668

# VQE with Transverse Ising Model Hamiltonian(4 Qubits) + LINEAR Entangelment Ansatz Using COBYLA Optimazer

In [11]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 4
Entangelment = "linear"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = cobyla_optimazer
VQE(optimazer, initial_thetas)
    

The Hamiltonian running VQE on is:
0.0 * IIII
+ 0.08936700311619616 * XIII
+ 0.05821528908798457 * IXII
+ 0.8222947954335467 * ZZII
+ 0.1523566328069712 * IIXI
+ 0.41740717004270533 * ZIZI
+ 0.3731456072626391 * IZZI
+ 0.3788318574112455 * IIIX
+ 0.8622534518076894 * ZIIZ
+ 0.29594019557918694 * IZIZ
+ 0.7141512435242652 * IIZZ
total expectation value:
0.43504767642184333
total expectation value:
0.7215397090310596
total expectation value:
0.2922273730359378
total expectation value:
0.274357580355968
total expectation value:
0.22957077153493421
total expectation value:
0.2503235087228579
total expectation value:
0.14486640376040522
total expectation value:
0.11715187301638347
total expectation value:
0.27636892487571363
total expectation value:
0.4153876573442373
total expectation value:
0.26219053959173894
total expectation value:
0.4285043156707842
total expectation value:
0.2783673594148238
total expectation value:
0.2950884025914533
total expectation value:
0.2720185968173588
total

# VQE with Transverse Ising Model Hamiltonian(4 Qubits) + FULL Entangelment Ansatz Using COBYLA Optimazer

In [12]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 4
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = cobyla_optimazer
VQE(optimazer, initial_thetas)

The Hamiltonian running VQE on is:
0.0 * IIII
+ 0.08936700311619616 * XIII
+ 0.05821528908798457 * IXII
+ 0.8222947954335467 * ZZII
+ 0.1523566328069712 * IIXI
+ 0.41740717004270533 * ZIZI
+ 0.3731456072626391 * IZZI
+ 0.3788318574112455 * IIIX
+ 0.8622534518076894 * ZIIZ
+ 0.29594019557918694 * IZIZ
+ 0.7141512435242652 * IIZZ
total expectation value:
0.7388653474701138
total expectation value:
1.1483001791875713
total expectation value:
0.7393147054372541
total expectation value:
0.6108507855537183
total expectation value:
0.45520439966822307
total expectation value:
0.39687201452460746
total expectation value:
0.24299121905969778
total expectation value:
0.2576896078205446
total expectation value:
0.40309098884102734
total expectation value:
0.24642894105908864
total expectation value:
0.5536979334586072
total expectation value:
0.2474601093838435
total expectation value:
0.3924232453147735
total expectation value:
0.3942356382606863
total expectation value:
0.34692164087118177
tota

# VQE with Transverse Ising Model Hamiltonian(3 Qubits) + LINEAR Entangelment Ansatz

In [13]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 3
Ham = get_hamiltonian("Ising Model")
Entangelment = "linear"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)
    

The Hamiltonian running VQE on is:
0.0 * III
+ 0.2961191691357784 * XII
+ 0.4182579829363764 * IXI
+ 0.45829279051191274 * ZZI
+ 0.27104265984020337 * IIX
+ 0.7745365236740532 * ZIZ
+ 0.1931048175891643 * IZZ
total expectation value:
0.8159993517706782
total expectation value:
0.7598802892975032
total expectation value:
0.7906661412019753
total expectation value:
0.8087948128860848
total expectation value:
0.738661526075336
total expectation value:
0.8042644999906411
total expectation value:
0.8061533855072072
total expectation value:
0.8389845486215579
total expectation value:
0.7966991763930528
total expectation value:
0.7942163999193789
total expectation value:
0.7884234190422847
total expectation value:
0.8006280833158941
total expectation value:
0.7590631891927355
total expectation value:
0.8263672829890208
total expectation value:
0.7922167127908395
total expectation value:
0.8090828844009816
total expectation value:
0.7898047077761845
total expectation value:
0.7222771007074581


total expectation value:
0.7774734357023668
total expectation value:
0.807400383517721
total expectation value:
0.784785396443825
total expectation value:
0.7662361363528288
total expectation value:
0.7296262787923861
total expectation value:
0.84039863596573
total expectation value:
0.8169938896860525
total expectation value:
0.8359231204333871
total expectation value:
0.7775808575592315
total expectation value:
0.7895970006919782
total expectation value:
0.7754318083723786
total expectation value:
0.8311649910974337
total expectation value:
0.8240752211775159
total expectation value:
0.784163597642107
total expectation value:
0.7827616326062796
total expectation value:
0.8365601378257463
total expectation value:
0.7844973249289281
total expectation value:
0.7717922954153347
total expectation value:
0.7762460956676613
total expectation value:
0.8178796832936612
total expectation value:
0.8295059793255857
total expectation value:
0.8157696459336435
total expectation value:
0.8223779530

total expectation value:
0.8264037654791834
total expectation value:
0.7557930149422697
total expectation value:
0.8194276099807238
total expectation value:
0.7731347402300826
total expectation value:
0.7553665174436015
total expectation value:
0.8112308946097457
total expectation value:
0.8310967489750876
total expectation value:
0.8261332739839174
total expectation value:
0.8219180693775752
total expectation value:
0.7496967439596375
total expectation value:
0.793620168185251
total expectation value:
0.7967188137168917
total expectation value:
0.793705990327228
total expectation value:
0.8490874460559209
total expectation value:
0.8393428039978802
total expectation value:
0.7779457251900574
total expectation value:
0.7617601694399906
total expectation value:
0.7850248631602146
total expectation value:
0.8286150114795026
total expectation value:
0.814391757606822
total expectation value:
0.8024275990276911
total expectation value:
0.7630969886357689
total expectation value:
0.81818682

# VQE with Transverse Ising Model Hamiltonian(3 Qubits) + FULL Entangelment Ansatz

In [14]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 3
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)

The Hamiltonian running VQE on is:
0.0 * III
+ 0.2961191691357784 * XII
+ 0.4182579829363764 * IXI
+ 0.45829279051191274 * ZZI
+ 0.27104265984020337 * IIX
+ 0.7745365236740532 * ZIZ
+ 0.1931048175891643 * IZZ
total expectation value:
1.1250812226868394
total expectation value:
1.1531988939110418
total expectation value:
1.0992022745790406
total expectation value:
1.1134694103922493
total expectation value:
1.1137537980272556
total expectation value:
1.1848362504542664
total expectation value:
1.1148772845198582
total expectation value:
1.0951085035344323
total expectation value:
1.123755486821317
total expectation value:
1.1183151357015755
total expectation value:
1.1268517915759377
total expectation value:
1.1460015865691306
total expectation value:
1.1512175216554406
total expectation value:
1.1274131673955852
total expectation value:
1.1446115759339646
total expectation value:
1.0361760335830592
total expectation value:
1.1069807629721216
total expectation value:
1.0874744864340025


total expectation value:
1.0783368759675336
total expectation value:
1.1358017836625347
total expectation value:
1.1725398549468984
total expectation value:
1.1581962064905726
total expectation value:
1.1366896552263208
total expectation value:
1.1275759535611745
total expectation value:
1.0991792368481232
total expectation value:
1.1046384904383968
total expectation value:
1.1388952756076705
total expectation value:
1.1229162200560379
total expectation value:
1.0998592334200203
total expectation value:
1.134131688560431
total expectation value:
1.139827596057609
total expectation value:
1.1722910787316487
total expectation value:
1.1499539826033844
total expectation value:
1.126317588747593
total expectation value:
1.103516609869507
total expectation value:
1.176477032390511
total expectation value:
1.1362499964412192
total expectation value:
1.1247924163186356
total expectation value:
1.1287365418321182
total expectation value:
1.1207470821648509
total expectation value:
1.1534342253

total expectation value:
1.1606569635594488
total expectation value:
1.07988629301318
total expectation value:
1.1372965189102096
total expectation value:
1.1213920866051692
total expectation value:
1.1060150563143174
total expectation value:
1.093632219376794
total expectation value:
1.1121430869291333
total expectation value:
1.177892893566079
total expectation value:
1.1242783228465942
total expectation value:
1.1293841911742382
total expectation value:
1.1309801350622142
total expectation value:
1.0833027123876928
total expectation value:
1.1099517823774345
total expectation value:
1.0909169242566006
total expectation value:
1.1417549058034167
total expectation value:
1.1079334762511752
total expectation value:
1.1206710209022288
total expectation value:
1.1573886993574771
total expectation value:
1.1193889990695516
total expectation value:
1.0980390414062955
total expectation value:
1.094341782059568
total expectation value:
1.140122332169564
total expectation value:
1.11937791573

# VQE with Transverse Ising Model Hamiltonian(3 Qubits) + LINEAR Entangelment Ansatz Using COBYLA Optimazer

In [15]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 3
Entangelment = "linear"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = cobyla_optimazer
VQE(optimazer, initial_thetas)
    

The Hamiltonian running VQE on is:
0.0 * III
+ 0.2961191691357784 * XII
+ 0.4182579829363764 * IXI
+ 0.45829279051191274 * ZZI
+ 0.27104265984020337 * IIX
+ 0.7745365236740532 * ZIZ
+ 0.1931048175891643 * IZZ
total expectation value:
0.7986836449309508
total expectation value:
1.2165224842301947
total expectation value:
0.7088782830087405
total expectation value:
0.5556885927805056
total expectation value:
0.5321854506700113
total expectation value:
0.7920753790844238
total expectation value:
0.5233032810158502
total expectation value:
0.5687057609046482
total expectation value:
0.7995914785953266
total expectation value:
0.8078103399748472
total expectation value:
0.5378854934275751
total expectation value:
0.6739164902655842
total expectation value:
0.7200661271413353
total expectation value:
0.8581981876558685
total expectation value:
0.7163058013359982
total expectation value:
0.5411545337867926
total expectation value:
0.4249704681916246
total expectation value:
0.571975640643686


# VQE with Transverse Ising Model Hamiltonian(3 Qubits) + FULL Entangelment Ansatz Using COBYLA Optimazer

In [16]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 3
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = cobyla_optimazer
VQE(optimazer, initial_thetas)

The Hamiltonian running VQE on is:
0.0 * III
+ 0.2961191691357784 * XII
+ 0.4182579829363764 * IXI
+ 0.45829279051191274 * ZZI
+ 0.27104265984020337 * IIX
+ 0.7745365236740532 * ZIZ
+ 0.1931048175891643 * IZZ
total expectation value:
1.1362634412004649
total expectation value:
1.1291744584764087
total expectation value:
1.00173180755495
total expectation value:
0.842674533854998
total expectation value:
0.8276578052060607
total expectation value:
1.1994710654411194
total expectation value:
0.8865652956874376
total expectation value:
0.5023917959038929
total expectation value:
1.0718753162389025
total expectation value:
0.5708067708133557
total expectation value:
0.5974583811783857
total expectation value:
0.5501182541054471
total expectation value:
0.7343427927036413
total expectation value:
0.9064431470240458
total expectation value:
0.6923315655849697
total expectation value:
0.6617566203068468
total expectation value:
0.6476234710922428
total expectation value:
0.6400147169346215
to

# VQE with Transverse Ising Model Hamiltonian(2 Qubits) + LINEAR Entangelment Ansatz

In [17]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 2
Ham = get_hamiltonian("Ising Model")
Entangelment = "linear"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)
    

The Hamiltonian running VQE on is:
0.0 * II
+ 0.13536117625203214 * XI
+ 0.9561784709605426 * IX
+ 0.7921163276668802 * ZZ
total expectation value:
1.0621447053655615
total expectation value:
1.0528620921507152
total expectation value:
1.0528620921507152
total expectation value:
1.0219200481012278
total expectation value:
1.0544091943531897
total expectation value:
1.0776157273903053
total expectation value:
1.0358439679234972
total expectation value:
1.031202661316074
total expectation value:
1.031202661316074
total expectation value:
1.063691807568036
total expectation value:
1.0729744207828822
total expectation value:
1.0605976031630873
total expectation value:
1.0544091943531897
total expectation value:
1.063691807568036
total expectation value:
1.0698802163779335
total expectation value:
0.9925251062542146
total expectation value:
1.0451265811383434
total expectation value:
1.006449026076484
total expectation value:
0.9847895952418427
total expectation value:
0.9956193106591634
to

total expectation value:
1.0559562965556641
total expectation value:
1.0250142525061765
total expectation value:
1.0373910701259716
total expectation value:
1.031202661316074
total expectation value:
1.0497678877457666
total expectation value:
1.088445442807626
total expectation value:
1.0327497635185483
total expectation value:
1.0760686251878309
total expectation value:
1.100822260427421
total expectation value:
1.0590505009606128
total expectation value:
1.0327497635185483
total expectation value:
1.0544091943531897
total expectation value:
1.0791628295927798
total expectation value:
1.0296555591135996
total expectation value:
1.046673683340818
total expectation value:
1.023467150303702
total expectation value:
1.0482207855432921
total expectation value:
1.068333114175459
total expectation value:
1.063691807568036
total expectation value:
1.0698802163779335
total expectation value:
1.026561354708651
total expectation value:
1.0327497635185483
total expectation value:
1.0544091943531

# VQE with Transverse Ising Model Hamiltonian(2 Qubits) + FULL Entangelment Ansatz

In [18]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 2
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)

The Hamiltonian running VQE on is:
0.0 * II
+ 0.13536117625203214 * XI
+ 0.9561784709605426 * IX
+ 0.7921163276668802 * ZZ
total expectation value:
0.9863366974443172
total expectation value:
1.1023693626298954
total expectation value:
1.0002606172665864
total expectation value:
1.0327497635185483
total expectation value:
1.0590505009606128
total expectation value:
1.0698802163779335
total expectation value:
1.0575033987581384
total expectation value:
1.0281084569111254
total expectation value:
1.0281084569111254
total expectation value:
1.0698802163779335
total expectation value:
1.0698802163779335
total expectation value:
1.0729744207828822
total expectation value:
1.0497678877457666
total expectation value:
1.0575033987581384
total expectation value:
1.0497678877457666
total expectation value:
1.0575033987581384
total expectation value:
1.051314989948241
total expectation value:
1.1410469176917546
total expectation value:
1.1611592463239215
total expectation value:
1.181271574956088

total expectation value:
1.0497678877457666
total expectation value:
1.0559562965556641
total expectation value:
1.0172787414938047
total expectation value:
1.0714273185804077
total expectation value:
1.0760686251878309
total expectation value:
1.0404852745309203
total expectation value:
1.0760686251878309
total expectation value:
1.023467150303702
total expectation value:
1.0791628295927798
total expectation value:
1.0219200481012278
total expectation value:
1.0327497635185483
total expectation value:
1.0389381723284459
total expectation value:
1.031202661316074
total expectation value:
1.0791628295927798
total expectation value:
1.0203729458987534
total expectation value:
1.0250142525061765
total expectation value:
1.031202661316074
total expectation value:
1.0157316392913303
total expectation value:
1.0667860119729846
total expectation value:
1.0544091943531897
total expectation value:
1.046673683340818
total expectation value:
1.046673683340818
total expectation value:
1.0760686251

# VQE with Transverse Ising Model Hamiltonian(2 Qubits) + LINEAR Entangelment Ansatz Using COBYLA Optimazer

In [19]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 2
Entangelment = "linear"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = cobyla_optimazer
VQE(optimazer, initial_thetas)
    

The Hamiltonian running VQE on is:
0.0 * II
+ 0.13536117625203214 * XI
+ 0.9561784709605426 * IX
+ 0.7921163276668802 * ZZ
total expectation value:
1.0110903326839071
total expectation value:
1.3282462841911542
total expectation value:
0.7527242648706864
total expectation value:
0.7635539802880071
total expectation value:
0.5949198402183002
total expectation value:
1.150329530906601
total expectation value:
0.7604597758830584
total expectation value:
0.540771263131697
total expectation value:
0.622767679862839
total expectation value:
0.5454125697391201
total expectation value:
0.8749453388661621
total expectation value:
0.8594743168414184
total expectation value:
0.503640810272312
total expectation value:
1.0915396472125747
total expectation value:
1.2029310057907296
total expectation value:
0.5206589344995302
total expectation value:
0.49126399265251697
total expectation value:
0.5222060367020045
total expectation value:
0.6923872789741858
total expectation value:
0.5871843292059283


# VQE with Transverse Ising Model Hamiltonian(2 Qubits) + FULL Entangelment Ansatz Using COBYLA Optimazer

In [20]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 2
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = cobyla_optimazer
VQE(optimazer, initial_thetas)

The Hamiltonian running VQE on is:
0.0 * II
+ 0.13536117625203214 * XI
+ 0.9561784709605426 * IX
+ 0.7921163276668802 * ZZ
total expectation value:
1.031202661316074
total expectation value:
1.3638296348480647
total expectation value:
0.7774779001102765
total expectation value:
0.7573655714781096
total expectation value:
0.5608835917638639
total expectation value:
1.150329530906601
total expectation value:
0.7558184692756352
total expectation value:
0.5546951829539664
total expectation value:
0.6320502930776852
total expectation value:
0.5794488181935564
total expectation value:
0.8440032948166746
total expectation value:
0.8161554551721358
total expectation value:
0.5191118322970558
total expectation value:
1.0358439679234972
total expectation value:
1.2137607212080503
total expectation value:
0.5113763212846839
total expectation value:
0.503640810272312
total expectation value:
0.697028585581609
total expectation value:
0.7403474472508915
total expectation value:
0.7016698921890321
t

# VQE with H2 Molecule Hamiltonian(2 Qubits) + LINEAR Entangelment Ansatz

In [21]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 2
Ham = get_hamiltonian("H2")
Entangelment = "linear"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)

The Hamiltonian running VQE on is:
-0.5053051899926562 * II
- 0.3277380754984016 * ZI
+ 0.15567463610622564 * ZZ
- 0.3277380754984016 * IZ
total expectation value:
-0.30661395204189035
total expectation value:
-0.32269677411330566
total expectation value:
-0.3070299998006731
total expectation value:
-0.32229669394540705
total expectation value:
-0.29317157003902616
total expectation value:
-0.3067259477770281
total expectation value:
-0.31034256267934984
total expectation value:
-0.2872025253012645
total expectation value:
-0.2917633056559391
total expectation value:
-0.3077501737827504
total expectation value:
-0.2958919964612972
total expectation value:
-0.31259896437053614
total expectation value:
-0.2950758685346159
total expectation value:
-0.2780969321827995
total expectation value:
-0.3153514743738747
total expectation value:
-0.2784329193882127
total expectation value:
-0.3174958803299232
total expectation value:
-0.0906562127040946
total expectation value:
-0.09670524379557613

total expectation value:
-0.3009969363000756
total expectation value:
-0.3070139580101393
total expectation value:
-0.2879066574928081
total expectation value:
-0.31109474604284504
total expectation value:
-0.3027732714696434
total expectation value:
-0.29965269067982425
total expectation value:
-0.3089183307053786
total expectation value:
-0.29112319222723126
total expectation value:
-0.3061658207020401
total expectation value:
-0.3117348594715529
total expectation value:
-0.3070620091820909
total expectation value:
-0.30078891242068423
total expectation value:
-0.2882586864887551
total expectation value:
-0.3185200321359959
total expectation value:
-0.3026772433253898
total expectation value:
-0.3207924756177159
total expectation value:
-0.2921154088515357
total expectation value:
-0.3081022027786973
total expectation value:
-0.29117116919953334
total expectation value:
-0.2981483981524835
total expectation value:
-0.31408728930699276
total expectation value:
-0.3208884295623199
tota

total expectation value:
-0.2898749747512336
total expectation value:
-0.2878266711390882
total expectation value:
-0.3022131443946555
total expectation value:
-0.28990698413265137
total expectation value:
-0.30416549406219684
total expectation value:
-0.3250171945673277
total expectation value:
-0.32621736087137376
total expectation value:
-0.2816975052945875
total expectation value:
-0.29104313167386175
total expectation value:
-0.3378194631414839
total expectation value:
-0.3179759468515416
total expectation value:
-0.3184560133731601
total expectation value:
-0.2904830787985234
total expectation value:
-0.3039414283922717
total expectation value:
-0.3242810529943662
The VQE result for the minimal energy of the Hamiltonian given is:
-0.3273056056399318
The actual minimal energy is:
(-1.0051067048832336+0j)
The differnce between the actual value and the algorithm result is:
(0.6778010992433019+0j)


# VQE with H2 Molecule Hamiltonian(2 Qubits) + FULL Entangelment Ansatz

In [22]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 2
Ham = get_hamiltonian("H2")
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)

The Hamiltonian running VQE on is:
-0.5053051899926562 * II
- 0.3277380754984016 * ZI
+ 0.15567463610622564 * ZZ
- 0.3277380754984016 * IZ
total expectation value:
-0.315207469257319
total expectation value:
-0.28099344730269354
total expectation value:
-0.29928461989334354
total expectation value:
-0.31949620696976655
total expectation value:
-0.2953319435859589
total expectation value:
-0.3219766001312283
total expectation value:
-0.26663890922889455
total expectation value:
-0.29946070859096663
total expectation value:
-0.32584929008489305
total expectation value:
-0.32688948368149945
total expectation value:
-0.30568575418042165
total expectation value:
-0.2880186532279458
total expectation value:
-0.2849941005823802
total expectation value:
-0.3060057737949508
total expectation value:
-0.29001897986778913
total expectation value:
-0.3474371321629853
total expectation value:
-0.2963240860106137
total expectation value:
-0.30739814478680333
total expectation value:
-0.34967764046293

total expectation value:
-0.3151754598759012
total expectation value:
-0.2830577927053726
total expectation value:
-0.2929635461596348
total expectation value:
-0.3164876961147347
total expectation value:
-0.2873304886272864
total expectation value:
-0.3130950726826884
total expectation value:
-0.3151914274667853
total expectation value:
-0.31743186156708747
total expectation value:
-0.29861257128286744
total expectation value:
-0.2988846139250945
total expectation value:
-0.3031413422561242
total expectation value:
-0.2970602275835751
total expectation value:
-0.29502789156231385
total expectation value:
-0.28865876665665363
total expectation value:
-0.29688421308560164
total expectation value:
-0.30832626844862243
total expectation value:
-0.3163756261799473
total expectation value:
-0.30938242963611295
total expectation value:
-0.32167254810758333
total expectation value:
-0.2973322702258022
total expectation value:
-0.3230008261369506
total expectation value:
-0.3058778104689289
to

# VQE with H2 Molecule Hamiltonian(2 Qubits) + LINEAR Entangelment Ansatz Using COBYLA Optimazer

In [23]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 2
Ham = get_hamiltonian("H2")
Entangelment = "linear"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = cobyla_optimazer
VQE(optimazer, initial_thetas)

The Hamiltonian running VQE on is:
-0.5053051899926562 * II
- 0.3277380754984016 * ZI
+ 0.15567463610622564 * ZZ
- 0.3277380754984016 * IZ
total expectation value:
-0.275600497230804
total expectation value:
-0.028501042957301492
total expectation value:
-0.40938383283248925
total expectation value:
-0.37672200617785706
total expectation value:
-0.636143736703547
total expectation value:
-0.5282373518906494
total expectation value:
-0.5454559505050269
total expectation value:
-0.579317646665107
total expectation value:
-0.5227957570496112
total expectation value:
-0.4626891877135619
total expectation value:
-0.7968286200979905
total expectation value:
-0.7368497914890142
total expectation value:
-0.7971806490939375
total expectation value:
-0.8662495484283924
total expectation value:
-0.8221299956184536
total expectation value:
-0.8627609710517421
total expectation value:
-0.8679618648351245
total expectation value:
-0.6677665569559669
total expectation value:
-0.9270285373731658
total

# VQE with H2 Molecule Hamiltonian(2 Qubits) + FULL Entangelment Ansatz Using COBYLA Optimazer

In [24]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 2
Ham = get_hamiltonian("H2")
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = cobyla_optimazer
VQE(optimazer, initial_thetas)

The Hamiltonian running VQE on is:
-0.5053051899926562 * II
- 0.3277380754984016 * ZI
+ 0.15567463610622564 * ZZ
- 0.3277380754984016 * IZ
total expectation value:
-0.32468113316226477
total expectation value:
-0.06465137278893451
total expectation value:
-0.38536350036558764
total expectation value:
-0.3949652759958546
total expectation value:
-0.6207810144145594
total expectation value:
-0.517835490124235
total expectation value:
-0.5810621950522059
total expectation value:
-0.5613464194893365
total expectation value:
-0.5394387061960144
total expectation value:
-0.4615530401723515
total expectation value:
-0.7708240027817793
total expectation value:
-0.726031881963817
total expectation value:
-0.7607742442109615
total expectation value:
-0.7381144959534435
total expectation value:
-0.6646289697801688
total expectation value:
-0.7730483950915477
total expectation value:
-0.773864523018229
total expectation value:
-0.6421295652283386
total expectation value:
-0.7924758636227073
total 

# VQE with H2 Molecule Hamiltonian(4 Qubits) + LINEAR Entangelment Ansatz

In [25]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 4
Ham = get_hamiltonian("H2")
Entangelment = "linear"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)

The Hamiltonian running VQE on is:
-0.8105479805373279 * IIII
+ 0.1721839326191554 * IIIZ
- 0.22575349222402372 * IIZI
+ 0.17218393261915543 * IZII
- 0.2257534922240237 * ZIII
+ 0.12091263261776627 * IIZZ
+ 0.16892753870087907 * IZIZ
+ 0.045232799946057826 * YYYY
+ 0.045232799946057826 * XXYY
+ 0.045232799946057826 * YYXX
+ 0.045232799946057826 * XXXX
+ 0.1661454325638241 * ZIIZ
+ 0.1661454325638241 * IZZI
+ 0.17464343068300453 * ZIZI
+ 0.12091263261776627 * ZZII
total expectation value:
-0.8143466865901946
total expectation value:
-0.839320907512741
total expectation value:
-0.8116404592489378
total expectation value:
-0.81456949715932
total expectation value:
-0.8032253627068638
total expectation value:
-0.8083000330211985
total expectation value:
-0.804839716889995
total expectation value:
-0.7883425172794034
total expectation value:
-0.7929242400360643
total expectation value:
-0.8119760426766358
total expectation value:
-0.8093575122970403
total expectation value:
-0.8084373436688

total expectation value:
-0.8124208354373444
total expectation value:
-0.8097874262146508
total expectation value:
-0.7744930031766848
total expectation value:
-0.8016967623949609
total expectation value:
-0.8284950683692703
total expectation value:
-0.8407481097548761
total expectation value:
-0.8209550750360523
total expectation value:
-0.817395400578689
total expectation value:
-0.8242715299444296
total expectation value:
-0.8139561078718938
total expectation value:
-0.8197036315836949
total expectation value:
-0.8109715675571653
total expectation value:
-0.8150781935383857
total expectation value:
-0.8125205835756065
total expectation value:
-0.8283222156410278
total expectation value:
-0.8254527610034789
total expectation value:
-0.8262772248747254
total expectation value:
-0.8035474364568672
total expectation value:
-0.8066209797451273
total expectation value:
-0.8115558062605974
total expectation value:
-0.7968539426300462
total expectation value:
-0.7982471036619252
total expec

total expectation value:
-0.8008732366788958
total expectation value:
-0.8209281518373738
total expectation value:
-0.8141701775579618
total expectation value:
-0.8122210023184092
total expectation value:
-0.8120555691842062
total expectation value:
-0.7956441307138431
total expectation value:
-0.7892698679248169
total expectation value:
-0.7942104360533775
total expectation value:
-0.8289391479739867
total expectation value:
-0.7798451284121166
total expectation value:
-0.8139032581323868
total expectation value:
-0.8350470312578897
total expectation value:
-0.7668153659756991
total expectation value:
-0.8205479584504519
total expectation value:
-0.7971056500528066
total expectation value:
-0.8069686271393889
total expectation value:
-0.7977244230797155
total expectation value:
-0.798742239703532
total expectation value:
-0.8149335896085984
total expectation value:
-0.8235440267675058
total expectation value:
-0.8186351954882299
total expectation value:
-0.8208648171691086
total expec

total expectation value:
-0.8416654179634049
total expectation value:
-0.8237868109311585
total expectation value:
-0.8184589832261692
total expectation value:
-0.8040779211147986
total expectation value:
-0.793234799382192
total expectation value:
-0.8072075652180679
total expectation value:
-0.8324350046716907
total expectation value:
-0.8213769813793222
total expectation value:
-0.8248493881395274
total expectation value:
-0.8444119446142097
total expectation value:
-0.8270656544707531
total expectation value:
-0.8259222061924395
total expectation value:
-0.831302115248912
total expectation value:
-0.8358021036346741
total expectation value:
-0.8455705404829537
total expectation value:
-0.8441857741898858
total expectation value:
-0.8166353066404695
total expectation value:
-0.8024486621292315
total expectation value:
-0.8117535538782797
total expectation value:
-0.8105567011877837
total expectation value:
-0.807057960345554
total expectation value:
-0.8199557777278927
total expecta

# VQE with H2 Molecule Hamiltonian(4 Qubits) + FULL Entangelment Ansatz

In [26]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 4
Ham = get_hamiltonian("H2")
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)

The Hamiltonian running VQE on is:
-0.8105479805373279 * IIII
+ 0.1721839326191554 * IIIZ
- 0.22575349222402372 * IIZI
+ 0.17218393261915543 * IZII
- 0.2257534922240237 * ZIII
+ 0.12091263261776627 * IIZZ
+ 0.16892753870087907 * IZIZ
+ 0.045232799946057826 * YYYY
+ 0.045232799946057826 * XXYY
+ 0.045232799946057826 * YYXX
+ 0.045232799946057826 * XXXX
+ 0.1661454325638241 * ZIIZ
+ 0.1661454325638241 * IZZI
+ 0.17464343068300453 * ZIZI
+ 0.12091263261776627 * ZZII
total expectation value:
-0.4903116049478488
total expectation value:
-0.4882795157633939
total expectation value:
-0.5072347890838815
total expectation value:
-0.49371355807413364
total expectation value:
-0.4808716000300765
total expectation value:
-0.4989052494019375
total expectation value:
-0.5033509408658922
total expectation value:
-0.5114407694576949
total expectation value:
-0.5067678791687175
total expectation value:
-0.49479052090589387
total expectation value:
-0.4861894289021039
total expectation value:
-0.5078586

total expectation value:
-0.5026928111922897
total expectation value:
-0.4886266569597953
total expectation value:
-0.49964597239267733
total expectation value:
-0.49541802556533987
total expectation value:
-0.5276988061644586
total expectation value:
-0.5192261336369541
total expectation value:
-0.5080512637484113
total expectation value:
-0.5060292470751944
total expectation value:
-0.4959712877763772
total expectation value:
-0.5369832599040962
total expectation value:
-0.4961005498522028
total expectation value:
-0.5056839325011475
total expectation value:
-0.5040766336709036
total expectation value:
-0.47945495247996217
total expectation value:
-0.5391416866462172
total expectation value:
-0.5161794169856909
total expectation value:
-0.4730731624204028
total expectation value:
-0.501215147117924
total expectation value:
-0.4957436397981508
total expectation value:
-0.5031182680084344
total expectation value:
-0.5084380065943008
total expectation value:
-0.47854247054931653
total e

total expectation value:
-0.5002945251775789
total expectation value:
-0.5091201101977638
total expectation value:
-0.5391580518668018
total expectation value:
-0.48218071127640505
total expectation value:
-0.48705748398569726
total expectation value:
-0.5051387968019933
total expectation value:
-0.5191981290653916
total expectation value:
-0.49973361261814603
total expectation value:
-0.522297074065209
total expectation value:
-0.5156757159646995
total expectation value:
-0.505035700924592
total expectation value:
-0.5104190848847237
total expectation value:
-0.5000259533987969
total expectation value:
-0.4969145540020449
total expectation value:
-0.5079506407405601
total expectation value:
-0.49516839431148235
total expectation value:
-0.49311898452331143
total expectation value:
-0.47091301583178946
total expectation value:
-0.4851093287712843
total expectation value:
-0.5128387730280269
total expectation value:
-0.4738662125480954
total expectation value:
-0.4917012598869682
total 

total expectation value:
-0.4923097156266266
total expectation value:
-0.5113391080203956
total expectation value:
-0.47219442913225884
total expectation value:
-0.5176080225435886
total expectation value:
-0.4971907171107953
total expectation value:
-0.4846353016497959
total expectation value:
-0.4859975704714777
total expectation value:
-0.49745972164556335
total expectation value:
-0.4935084259789493
total expectation value:
-0.5065510099219344
total expectation value:
-0.49377363401416097
total expectation value:
-0.48965820117224756
total expectation value:
-0.5087492843206066
total expectation value:
-0.4692966029265147
total expectation value:
-0.48722527369780194
total expectation value:
-0.5281812290492197
total expectation value:
-0.5164922963382415
total expectation value:
-0.4597471468210896
total expectation value:
-0.49076998895606616
total expectation value:
-0.46686812922014337
total expectation value:
-0.48712416840780415
total expectation value:
-0.5035922959127391
to

total expectation value:
-0.50292352982141
total expectation value:
-0.5138302502237301
total expectation value:
-0.49423023948012995
total expectation value:
-0.5283669629749668
total expectation value:
-0.5024584504214918
total expectation value:
-0.47017888870055735
total expectation value:
-0.4951023941243552
total expectation value:
-0.5110173821725537
total expectation value:
-0.49740896807575696
total expectation value:
-0.4942158784337282
total expectation value:
-0.4844799923256668
total expectation value:
-0.5151054834610526
total expectation value:
-0.5004451404375765
total expectation value:
-0.5090163903908379
total expectation value:
-0.4937390940207011
total expectation value:
-0.5083469524217632
total expectation value:
-0.5218806780418764
total expectation value:
-0.5096076246487992
total expectation value:
-0.525874383513677
total expectation value:
-0.4891087782765182
total expectation value:
-0.5079199240360145
total expectation value:
-0.49733137809543565
total exp

# VQE with H2 Molecule Hamiltonian(4 Qubits) + LINEAR Entangelment Ansatz Using COBYLA Optimazer

In [27]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 4
Ham = get_hamiltonian("H2")
Entangelment = "linear"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = cobyla_optimazer
VQE(optimazer, initial_thetas)

The Hamiltonian running VQE on is:
-0.8105479805373279 * IIII
+ 0.1721839326191554 * IIIZ
- 0.22575349222402372 * IIZI
+ 0.17218393261915543 * IZII
- 0.2257534922240237 * ZIII
+ 0.12091263261776627 * IIZZ
+ 0.16892753870087907 * IZIZ
+ 0.045232799946057826 * YYYY
+ 0.045232799946057826 * XXYY
+ 0.045232799946057826 * YYXX
+ 0.045232799946057826 * XXXX
+ 0.1661454325638241 * ZIIZ
+ 0.1661454325638241 * IZZI
+ 0.17464343068300453 * ZIZI
+ 0.12091263261776627 * ZZII
total expectation value:
-0.8269961304860847
total expectation value:
-0.7964967032537746
total expectation value:
-0.8004277267066418
total expectation value:
-0.7517843162097004
total expectation value:
-0.744950680752968
total expectation value:
-0.8116278976130104
total expectation value:
-0.7644918922231811
total expectation value:
-0.8138824055716499
total expectation value:
-0.8004780221640406
total expectation value:
-0.7445252244739705
total expectation value:
-0.804540839393508
total expectation value:
-0.85109581479

# VQE with H2 Molecule Hamiltonian(4 Qubits) + FULL Entangelment Ansatz Using COBYLA Optimazer

In [28]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 4
Ham = get_hamiltonian("H2")
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = cobyla_optimazer
VQE(optimazer, initial_thetas)

The Hamiltonian running VQE on is:
-0.8105479805373279 * IIII
+ 0.1721839326191554 * IIIZ
- 0.22575349222402372 * IIZI
+ 0.17218393261915543 * IZII
- 0.2257534922240237 * ZIII
+ 0.12091263261776627 * IIZZ
+ 0.16892753870087907 * IZIZ
+ 0.045232799946057826 * YYYY
+ 0.045232799946057826 * XXYY
+ 0.045232799946057826 * YYXX
+ 0.045232799946057826 * XXXX
+ 0.1661454325638241 * ZIIZ
+ 0.1661454325638241 * IZZI
+ 0.17464343068300453 * ZIZI
+ 0.12091263261776627 * ZZII
total expectation value:
-0.506701825683557
total expectation value:
-0.3277672479867155
total expectation value:
-0.3819756497353391
total expectation value:
-0.517057200951723
total expectation value:
-0.6406184363253087
total expectation value:
-0.6172482103373368
total expectation value:
-0.7113359686768232
total expectation value:
-0.7145548775653585
total expectation value:
-0.737777537324878
total expectation value:
-0.755095117173256
total expectation value:
-0.6435401483561749
total expectation value:
-0.7346112765685

# VQE with LiH Molecule Hamiltonian(4 Qubits) + LINEAR Entangelment Ansatz

In [29]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 4
Ham = get_hamiltonian("LiH")
Entangelment = "linear"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)

The Hamiltonian running VQE on is:
-7.49894690201071 * IIII
- 0.0029329964409502266 * XXYY
+ 0.0029329964409502266 * XYYX
+ 0.01291078027311749 * XZXI
- 0.0013743761078958677 * XZXZ
+ 0.011536413200774975 * XIXI
+ 0.0029329964409502266 * YXXY
- 0.0029329964409502266 * YYXX
+ 0.01291078027311749 * YZYI
- 0.0013743761078958677 * YZYZ
+ 0.011536413200774975 * YIYI
+ 0.16199475388004184 * ZIII
+ 0.011536413200774975 * ZXZX
+ 0.011536413200774975 * ZYZY
+ 0.12444770133137588 * ZZII
+ 0.054130445793298836 * ZIZI
+ 0.05706344223424907 * ZIIZ
+ 0.012910780273117487 * IXZX
- 0.0013743761078958677 * IXIX
+ 0.012910780273117487 * IYZY
- 0.0013743761078958677 * IYIY
+ 0.16199475388004186 * IZII
+ 0.05706344223424907 * IZZI
+ 0.054130445793298836 * IZIZ
- 0.013243698330265966 * IIZI
+ 0.08479609543670981 * IIZZ
- 0.013243698330265952 * IIIZ
total expectation value:
-7.461645653014872
total expectation value:
-7.469913210545506
total expectation value:
-7.448694880639133
total expectation value:
-7.

total expectation value:
-7.468297912506979
total expectation value:
-7.476998728020682
total expectation value:
-7.457527355472835
total expectation value:
-7.443035230642852
total expectation value:
-7.46697604640794
total expectation value:
-7.476068679876168
total expectation value:
-7.453087686639177
total expectation value:
-7.450625633603414
total expectation value:
-7.447591077602134
total expectation value:
-7.472058396666166
total expectation value:
-7.455815302546364
total expectation value:
-7.448261881533593
total expectation value:
-7.458572424641975
total expectation value:
-7.438735807266067
total expectation value:
-7.461193074536015
total expectation value:
-7.458163433640196
total expectation value:
-7.461386269850391
total expectation value:
-7.466687283169683
total expectation value:
-7.471970958585809
total expectation value:
-7.460307612051451
total expectation value:
-7.454443710039721
total expectation value:
-7.437516728290614
total expectation value:
-7.44094

total expectation value:
-7.445883756804553
total expectation value:
-7.455343727732171
total expectation value:
-7.457250606075582
total expectation value:
-7.446803533162194
total expectation value:
-7.4667264626806205
total expectation value:
-7.461548026318227
total expectation value:
-7.467981442371384
total expectation value:
-7.452978514913944
total expectation value:
-7.445466828171074
total expectation value:
-7.463329336998474
total expectation value:
-7.4474745591624725
total expectation value:
-7.476201931780303
total expectation value:
-7.4583620689897145
total expectation value:
-7.455512999868652
total expectation value:
-7.457234653655985
total expectation value:
-7.433930139546425
total expectation value:
-7.468797586970106
total expectation value:
-7.465751452019449
total expectation value:
-7.469353826625801
total expectation value:
-7.460630166702249
total expectation value:
-7.45777277285379
total expectation value:
-7.46461518851849
total expectation value:
-7.433

total expectation value:
-7.4439931991211195
total expectation value:
-7.458676929151547
total expectation value:
-7.460137576706257
total expectation value:
-7.468274186454644
total expectation value:
-7.451100736886424
total expectation value:
-7.460370890690603
total expectation value:
-7.454595549685836
total expectation value:
-7.444419919512598
total expectation value:
-7.458642748887542
total expectation value:
-7.4452301281778395
total expectation value:
-7.472230503332331
total expectation value:
-7.444713416760636
total expectation value:
-7.455282115412946
total expectation value:
-7.457883254855008
total expectation value:
-7.453741447181561
total expectation value:
-7.469361281477135
total expectation value:
-7.464594802457389
total expectation value:
-7.453583160757099
total expectation value:
-7.45500818171284
total expectation value:
-7.443906118260492
total expectation value:
-7.473353393945416
total expectation value:
-7.448957168494264
total expectation value:
-7.444

total expectation value:
-7.453496932233472
total expectation value:
-7.475380194978669
total expectation value:
-7.455346552397138
total expectation value:
-7.445840341622282
total expectation value:
-7.445756887732303
total expectation value:
-7.452147778391863
total expectation value:
-7.465071416293666
total expectation value:
-7.461256980253218
total expectation value:
-7.462936380281286
total expectation value:
-7.448206366835199
total expectation value:
-7.445140504828362
total expectation value:
-7.4753396085002
total expectation value:
-7.458521576816504
total expectation value:
-7.458809709347449
total expectation value:
-7.456370640153499
total expectation value:
-7.459829698736146
total expectation value:
-7.442953417623785
total expectation value:
-7.453711091801859
total expectation value:
-7.465551638639514
total expectation value:
-7.451810194939438
total expectation value:
-7.4508592226780355
total expectation value:
-7.469459943291491
total expectation value:
-7.45981

# VQE with LiH Molecule Hamiltonian(4 Qubits) + FULL Entangelment Ansatz

In [30]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 4
Ham = get_hamiltonian("LiH")
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)

The Hamiltonian running VQE on is:
-7.49894690201071 * IIII
- 0.0029329964409502266 * XXYY
+ 0.0029329964409502266 * XYYX
+ 0.01291078027311749 * XZXI
- 0.0013743761078958677 * XZXZ
+ 0.011536413200774975 * XIXI
+ 0.0029329964409502266 * YXXY
- 0.0029329964409502266 * YYXX
+ 0.01291078027311749 * YZYI
- 0.0013743761078958677 * YZYZ
+ 0.011536413200774975 * YIYI
+ 0.16199475388004184 * ZIII
+ 0.011536413200774975 * ZXZX
+ 0.011536413200774975 * ZYZY
+ 0.12444770133137588 * ZZII
+ 0.054130445793298836 * ZIZI
+ 0.05706344223424907 * ZIIZ
+ 0.012910780273117487 * IXZX
- 0.0013743761078958677 * IXIX
+ 0.012910780273117487 * IYZY
- 0.0013743761078958677 * IYIY
+ 0.16199475388004186 * IZII
+ 0.05706344223424907 * IZZI
+ 0.054130445793298836 * IZIZ
- 0.013243698330265966 * IIZI
+ 0.08479609543670981 * IIZZ
- 0.013243698330265952 * IIIZ
total expectation value:
-7.37774136037675
total expectation value:
-7.383698810329846
total expectation value:
-7.388331819603154
total expectation value:
-7.3

total expectation value:
-7.3735369056398605
total expectation value:
-7.367937564923453
total expectation value:
-7.3653023564194635
total expectation value:
-7.379481178077139
total expectation value:
-7.3938052990532865
total expectation value:
-7.377904334421543
total expectation value:
-7.394171933152272
total expectation value:
-7.392859179153629
total expectation value:
-7.377433121291741
total expectation value:
-7.373924788426503
total expectation value:
-7.372111071674721
total expectation value:
-7.3847883978875215
total expectation value:
-7.389669511155328
total expectation value:
-7.3840036671033396
total expectation value:
-7.380636562196312
total expectation value:
-7.382029125650456
total expectation value:
-7.3985810405237595
total expectation value:
-7.377478555303124
total expectation value:
-7.377161458684378
total expectation value:
-7.384285603560728
total expectation value:
-7.392050650511733
total expectation value:
-7.394892282930705
total expectation value:
-

total expectation value:
-7.381007666623767
total expectation value:
-7.377820703995937
total expectation value:
-7.39035745172205
total expectation value:
-7.394149711574688
total expectation value:
-7.391125580975936
total expectation value:
-7.374955824195232
total expectation value:
-7.375030529686399
total expectation value:
-7.390668908509908
total expectation value:
-7.392706367568469
total expectation value:
-7.373504148387581
total expectation value:
-7.394859118426247
total expectation value:
-7.396482247248977
total expectation value:
-7.381690849663533
total expectation value:
-7.384084850414885
total expectation value:
-7.390706255634149
total expectation value:
-7.3956791264871065
total expectation value:
-7.395631097810808
total expectation value:
-7.382540485757279
total expectation value:
-7.376226850983544
total expectation value:
-7.4027332381444895
total expectation value:
-7.389574629441551
total expectation value:
-7.377688640849233
total expectation value:
-7.397

# VQE with LiH Molecule Hamiltonian(4 Qubits) + LINEAR Entangelment Ansatz Using COBYLA Optimazer

In [31]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 4
Ham = get_hamiltonian("LiH")
Entangelment = "linear"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = cobyla_optimazer
VQE(optimazer, initial_thetas)

The Hamiltonian running VQE on is:
-7.49894690201071 * IIII
- 0.0029329964409502266 * XXYY
+ 0.0029329964409502266 * XYYX
+ 0.01291078027311749 * XZXI
- 0.0013743761078958677 * XZXZ
+ 0.011536413200774975 * XIXI
+ 0.0029329964409502266 * YXXY
- 0.0029329964409502266 * YYXX
+ 0.01291078027311749 * YZYI
- 0.0013743761078958677 * YZYZ
+ 0.011536413200774975 * YIYI
+ 0.16199475388004184 * ZIII
+ 0.011536413200774975 * ZXZX
+ 0.011536413200774975 * ZYZY
+ 0.12444770133137588 * ZZII
+ 0.054130445793298836 * ZIZI
+ 0.05706344223424907 * ZIIZ
+ 0.012910780273117487 * IXZX
- 0.0013743761078958677 * IXIX
+ 0.012910780273117487 * IYZY
- 0.0013743761078958677 * IYIY
+ 0.16199475388004186 * IZII
+ 0.05706344223424907 * IZZI
+ 0.054130445793298836 * IZIZ
- 0.013243698330265966 * IIZI
+ 0.08479609543670981 * IIZZ
- 0.013243698330265952 * IIIZ
total expectation value:
-7.450646629418902
total expectation value:
-7.410578039411079
total expectation value:
-7.435104513357759
total expectation value:
-7.

# VQE with LiH Molecule Hamiltonian(4 Qubits) + FULL Entangelment Ansatz Using COBYLA Optimazer

In [32]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 4
Ham = get_hamiltonian("LiH")
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = cobyla_optimazer
VQE(optimazer, initial_thetas)

The Hamiltonian running VQE on is:
-7.49894690201071 * IIII
- 0.0029329964409502266 * XXYY
+ 0.0029329964409502266 * XYYX
+ 0.01291078027311749 * XZXI
- 0.0013743761078958677 * XZXZ
+ 0.011536413200774975 * XIXI
+ 0.0029329964409502266 * YXXY
- 0.0029329964409502266 * YYXX
+ 0.01291078027311749 * YZYI
- 0.0013743761078958677 * YZYZ
+ 0.011536413200774975 * YIYI
+ 0.16199475388004184 * ZIII
+ 0.011536413200774975 * ZXZX
+ 0.011536413200774975 * ZYZY
+ 0.12444770133137588 * ZZII
+ 0.054130445793298836 * ZIZI
+ 0.05706344223424907 * ZIIZ
+ 0.012910780273117487 * IXZX
- 0.0013743761078958677 * IXIX
+ 0.012910780273117487 * IYZY
- 0.0013743761078958677 * IYIY
+ 0.16199475388004186 * IZII
+ 0.05706344223424907 * IZZI
+ 0.054130445793298836 * IZIZ
- 0.013243698330265966 * IIZI
+ 0.08479609543670981 * IIZZ
- 0.013243698330265952 * IIIZ
total expectation value:
-7.386308414192465
total expectation value:
-7.329217884505318
total expectation value:
-7.377971682593789
total expectation value:
-7.

# VQE with H2 Molecule Hamiltonian(2 Qubits) + FULL Entangelment Ansatz Using COBYLA Optimazer on REAL HARDWAWE

In [34]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = real_backend

QUBITS_NUM = 2
Ham = get_hamiltonian("H2")
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = cobyla_optimazer
VQE(optimazer, initial_thetas)

The Hamiltonian running VQE on is:
-0.5053051899926562 * II
- 0.3277380754984016 * ZI
+ 0.15567463610622564 * ZZ
- 0.3277380754984016 * IZ


ibmqjob._wait_for_completion:ERROR:2022-07-30 01:03:14,102: Maximum retries exceeded: Error checking job status due to a network error.


IBMQJobApiError: 'Error checking job status due to a network error: "HTTPSConnectionPool(host=\'api-qcon.quantum-computing.ibm.com\', port=443): Max retries exceeded with url: /api/Network/ibm-q/Groups/open/Projects/main/Jobs/62e450c43e2c28bca1bd3449/status/v/1 (Caused by NewConnectionError(\'<urllib3.connection.HTTPSConnection object at 0x000001F67F53D0A0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed\'))"'